In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.decomposition import TruncatedSVD
from cut_sentence import segment

In [2]:
content = pd.read_csv('./movie_comments.csv', encoding='utf-8')
content = content.dropna(axis=0,how='any')
# print(len(content),content)

In [3]:
model = Word2Vec.load("wiki.model")
word_vectors = model.wv
del model

In [4]:
import random
#1. 预处理---分词
data=[]
for item in content.iterrows():
#     print(item[1]['comment'])
    words = segment(item[1]['comment'],type='arr')
#     print(words)
    data.append({'comment':item[1]['comment'],'star':item[1]['star'],'words':words})

#2. 预处理---分解为训练集/验证集/测试集
total_size = len(data)
print(len(data))
train_size = int(0.7*total_size)
print(train_size)
valid_size = int(0.2*total_size)
test_size = int(0.1*total_size)
print("use train_size: %d, valid_size: %d, test_size: %d" % (train_size,valid_size,test_size))

random.shuffle(data)
train_set = data[:train_size]
valid_set = data[train_size:train_size+valid_size]
test_set = data[(train_size+valid_size):train_size+valid_size+test_size]

# def sentence_embedding(sentence,vocabulary):
#     pass

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\webberg\AppData\Local\Temp\jieba.cache
Loading model cost 0.906 seconds.
Prefix dict has been built succesfully.


130454
91317
use train_size: 91317, valid_size: 26090, test_size: 13045


In [5]:
from collections import defaultdict
# 尝试采用SIF sentence embedding(https://www.colern.com/paper/get?paperid=1bcb3f85-45ba-4f21-9609-4a598eb32cec)

# 1.计算词频
# unigram probability
p_w=defaultdict(int)
train_words_count  = 0

for item in train_set:
    for word in item['words']:
        p_w[word]+=1
        train_words_count += 1
print(p_w['英雄'])

786


In [6]:
# 初始句向量
sentence_vectors=[]
alpha = 0.001
for sentence in train_set:
    sentence_len = len(sentence['words'])
#     print(sentence_len)
#         continue
#     print(sentence['words'])
    sentence_vec=np.zeros(128)
    for word in sentence['words']:
        if word in word_vectors.vocab:
            sentence_vec += word_vectors[word]*(alpha/(alpha+p_w[word]/train_words_count))
        # 抛弃UNK词,看看效果
    
    if sentence_len==0:
        print(sentence['comment'])
        sentence_vectors.append(None)
    else:
        sentence_vectors.append(sentence_vec/sentence_len)

Wonderful always~
Guide of Awkward Running
There's a front seat and a back seat, and a window in between.
8.4/10
paul Walker
The time live to die.
OK
2012.11.12 18:07
i like it
x
★★★☆
r u kidding me
There is even no TAKEN this time!
I hate pink...
you are my best adventure
4.5
SO COOL
justin timberlake!!!!
only unfulfilled love can be romantic
···
You are not an asshole, but you are trying to be.
hard 2 say
sadness streams while watching it on train
x
★★★★
2006.10.3
20070721 8/8.3(47059) #96
[design to work---die to work ]
Hey dad, I love you so much!
。。
Nyoron～
x
didn't get it ><
9.5/10
144
RIP
15.11.12cinema
“Knowing what you've got,comma; knowing what you need,comma; knowing what you can do without,dash."
If you are big on visuals, styles and details. This  is definately the one to watch. But storyline is still a long way away from Sin City.
=。=
tom's ugly hair
http://www.tudou.com/programs/view/GRmAd2vqr98/
It’s not who you are underneath, but it's what you do that defines you
This

Depp is A GOD.
2-13   2009
I feel lonely
Life is not the amount of breaths you take, it's the moments that take your breath away.
“The bright lure of freedom diminishes your life's joy in a mad scramble for power, for identity.”
。。
Cheers darling
12
HIGH！
........
he's gonna show u the magic
2007.11.15 CCTV6 home
I am Groot.
Brilliant,but...
JonathanDayton ValerieFaris
clfl1tbu  tlfhfcd
"Such a nice car"
4.5
james fucking cool danny fucking cool
Love is all around....
All Craziness in America ...
-  -
TV SHOW
sunshine!!!!!!
4!
20100812 1915 The Grand Cinema
BRAVO!BINGO!
11
3.5
2001
★★★★★★★★★★
Boring
My guilty pleasure!
Charming
I'm sorry Mr. Bay. I tried. I really tried.
what the hell
A waste of time
~~
very violent
a bit mo lei tau
somehow it's very early 90s...
9/9
so far so good
balalala
Col
1 26
2013.3.26 14:34
lovely story
lai.lai.lai.lai...
20120906
@20120413
http://v.youku.com/v_show/id_XOTgxMzE0NjQ=.html
so,love
。。。
so hot ,berry
remember the titans
A love letter to Keanu Reeve

★★☆
http://movie.douban.com/review/4291637/
he`s just not that into me!
Summer is a bitch
kalafina
x
The Beautiful World
「For the first time in my life, I got people respecting me. Up in Hinkley, I walk into a room and everyone shuts up to hear if I've got something to say. I never had that before. Ever.  Please, don't ask me to give it up.」
MBA course case - Leadership: coorditation and role definition
waltz for a night,moving
i wait.
If time could be traveled, I still wish you are there.
dream
7.5/8.2
Cool guy never look back
2006-10-10
So it's love~~~
wonderful !!!
the little boy is cute
x
I love the split-screen, James is a hot piece of Dean!
panzer vor!!!
dreamily
91
6.5/10
Ownship..
2012.07.19
2011.09.18
8/10
two tigers in one maountan"
Baby~
MLGB....
But you knew there would always be the spring, as you knew the river would flow again after it was frozen. When the cold rains kept on and killed the spring, it was as though a young person had died for no reason.
for Paul.
........

boring
http://www.tudou.com/programs/view/Lu4PJfIu0Og
This is just awesome! i love you Gru!
https://www.douban.com/people/hitchitsch/status/1889997185/
impressive match
x
I like you just as you are.
8.0
http://www.tudou.com/playlist/id/2629201/
I never forgot that she did love me...even if she did...all the time..all the time...all...the time...
EXM？！！！！！！！
old dogs' carnival
Views of Paris seen from a typical foreigners (or for foreigners) point of view. Some stories are good, some not.
7.0/7.0
1--1.1--1--2--3
4K restoration、35mm
follow your heart. good luck, son
x
5 stars, without any doubts.
waste my time
I wish I knew how to quit you.
It's not a movie.It's a life of everyone.
BGM！
sucks
...
62090
Nyoron☆ ~
Its not just a game
Rock & Roll save the Federation.
17.04.09@cinema
8.5/8.3
I got the same tensions when I tried to pass the security with my fake id to get into bars... -_-
undoubtably the best science fiction ever!
CCTV6
You got a dream, you gotta protect it. Don't let your dr

Lost
COOL
x
He's just not that into you.
Views of Paris seen from a typical foreigners (or for foreigners) point of view. Some stories are good, some not.
Anyone can cook! :)
You Are Not Alone
a mes parents.
sucks
A semi-decent whitewashed LSD trip.
- Who is the judge? 
- The judge is God. 
- Why is he God? 
- Because he decides who wins or loses not my opponent.
- Who is your opponent?
- He doesn't exist.
- Why does he not exist? 
- Because he is a mute dissenting voice the tr
8.1
I'll miss you
Mr. President  파이팅！
2007.6.26
99
......
what a cute couple!!
Best film ever
very funny
X  93008
30
run ,oscar
Blake Lively
2001.10.24 vcd
if Liberatore met Moebius and could make an anime... it might look like this. pretty anarchic.
All of life is an act of letting go,but what hurts the most is not taking a moment to say goodbye.
Megryan ！
the worst storyline, the worst music, the worst editing, the worst acting. a bunch of WTF.
It's a love story that I won't forget.
！
2008 9 13
nice but horrib

7.5/7.6
it's not a car, it's a war machine.
7.5/10
The first movie I saw in the theater
escape
i had a crush on that little dude....
It's all a lie.
life is like a box of chocolate.
x
If you have that much money to spend, put it into SAVING THIS PLANET first.
future gazer........!
u got me
oceans 。。。
pixar
"The basic principle is.... there are none"
No one will surpass American animation film makers.
7.2/7.1
★★★★☆
the worst option is to do nothing?
Having dreams is what makes life tolerable.
that's what i want
You need to call it. I can't call it for you. It wouldn't be fair
She is definitely something~
080121
anhong~~~
4.5
3
2012.12.25
5.1
only if the camera stays a bit more still
agsokrdt b7ptinfa
6.8/7.3
2017.7.26 17:06
Just love football for no reason. (Now I look back, yes, there is a reason. Ben. I didn't understand how much I liked him until later on. 8/10/2016.)
oh i like Sonny.
do you have a soulmate?
I've wanted: to walk in such a place with you, with friends, on the earth wi

The best thing you can do is to find somebody who loves you for exactly what you are, good mood, bad mood; beautiful, ugly.
8.0/7.2
Will go see it on Broadway.
2008-1-21 23:34:48 8/6.0(188)
best “made in HK" in years
···
8.6/10
/
bullshit
2001
9.0/10
drama
We do what we have to do in order to do what we want to do.
2006.8.4
84/67
Unchained Melody
Cool!
http://www.tudou.com/programs/view/wjyK0hEfwDI/
Robert Downey Jr.
Kim:Hold me...    Howard:No...
20100716 jtl
Shutter.Island.2010.BDRip.XviD-iMBT
So fucking sexy
New York, NY!
right!there is nothing wrong with homosexual!
ARE U ENJOY THE TIME OF EVE?
Love is murmuring ur name when sorrow came. Love is searching for the perfect word when writing a letter to u. Love is surviving during the war to come back to you. It's just so hard to find the right one and you know he's the only one.
he's so cute
X  93008
http://v.youku.com/v_show/id_XODc0MTI=.html
shame
CINEMATHEQUE  220208
SophieMarceau
。。。。
moon river, audrey hepburn
With great power, 

crazy jack
431M  .Bluray.X264.2Audio.AAC.HALFCD3-NORM.mkv
He was a friend of mine.
faint
TVT
..
Hope is a good thing and maybe the best of things. And no good thing ever dies
x
I LOVE YOU TOMMY T___T
9
OLYMPIAN CITY  241208
HOHOHOHOHO
True end
anne
Q stands for CUTE; dreadful script; villain's motive not at all convincing; typecasting of Waltz, Scott, Bellucci and Seydoux leads no surprise in storytelling; absolutely no chemistry between Craig and Seydoux
Shutter.Island.2010.BDRip.XviD-iMBT
all-time favorite.
2006.10.3
X  75876
PS...
huge got great figure
Hulk...Smash！！！
2012.12.14 22:53
Anyone can cook! :)
Depp。。~
[design to work---die to work ]
Great.
007
4.5
I hate my life, but I love this planet
。。。
···
GC~
A story between a tough guy and a cute little girl~
GATEWAY  201111
More than you read……
who TM care.
Cool..
2006//OVA
8.5/8.2
life is a miracle, and we all deserve a dream.
twice.....
Harry
8
http://so.youku.com/search_video/q_%E8%A5%BF%E8%A5%BF%E9%87%8C%E7%9A%84%E7%BE%8E%E4%B8

In [7]:
# print(len(sentence_vectors))
# print(train_set[0])
# 剔除表情文字及其他长度为0的句子
new_train_set = pd.DataFrame({'sentence':sentence_vectors,'star':[item['star'] for item in train_set]})
print(new_train_set.shape)
new_train_set = new_train_set.dropna(axis=0,how='any')
print(new_train_set.shape)

(91317, 2)
(88786, 2)


In [21]:
svd = TruncatedSVD(n_components=1,n_iter=5,random_state=0)
#将所有求得的初始句向量做成矩阵
print(new_train_set.shape[0],new_train_set.shape[1])
sentence_x = np.zeros((new_train_set.shape[0],len(sentence_vectors[0])))
for i in range(new_train_set.shape[0]):
    sentence_x[i,:]=new_train_set['sentence'].iloc[i]
print(sentence_x)
svd.fit(sentence_x)
pc = svd.components_
print(pc.shape)

88786 2
[[ 1.13976297e-02  2.21515916e-01 -1.66032887e-01 ...  2.01061898e-01
   2.56954806e-01  4.38383334e-02]
 [-1.25314817e-01 -1.28219772e-01  2.69481500e-01 ...  2.91126649e-01
  -5.89409818e-02 -2.71296141e-02]
 [-1.76015206e-01  5.01424372e-02 -7.51993358e-01 ...  2.55042052e+00
  -4.19480562e-01  3.50684822e-01]
 ...
 [ 9.63192731e-01 -2.37527117e-03 -8.58185212e-01 ... -4.99979953e-01
   5.42776604e-01  8.31814100e-01]
 [ 4.39223979e-01 -1.89047455e-01 -4.52335507e-01 ...  4.19947341e-01
   4.14690280e-01  1.77541549e-01]
 [ 1.35944319e-01 -3.62703098e-02 -1.96661580e-01 ...  1.98329991e-01
   4.89678552e-01  2.09828971e-01]]
(1, 128)


In [23]:
#计算每个句子的句向量
embed = sentence_x-sentence_x.dot(pc.transpose()) * pc
print(embed.shape)

(88786, 128)


In [26]:
print(train_set[5])
#查看句向量在词向量空间中的临近词
word_vectors.similar_by_vector(embed[5])

{'comment': '还不错啦，爱情故事，风景很美，主角们很迷人！', 'star': '3', 'words': ['还', '不错', '啦', '爱情故事', '风景', '很', '美', '主角', '们', '很', '迷人']}


[('风景', 0.5595492720603943),
 ('爱情故事', 0.5577840805053711),
 ('如诗如画', 0.5529738664627075),
 ('如画', 0.5516956448554993),
 ('绚樱', 0.5488759279251099),
 ('风情', 0.5368644595146179),
 ('田园风光', 0.5336508750915527),
 ('景色', 0.5288260579109192),
 ('童话', 0.5155296921730042),
 ('主人翁', 0.5098251104354858)]